## DeepFool Algorithm

For a given classifier and example, the algorithm is set to compute the minimal perturbation that is sufficient to change the estimated label. For the remainder of the section, the following notation will be used:

- $f$: a given classifier that outputs a vector with the probability distribution for the classification associated with its probability index.

- $x$: a given example.

- $X$: the domain of the examples.

- $T$: the domain of test examples available.

- $k$: a possible classification of the considered problem. Thus, the probability of the classification of an example $x$ to be $k$ is $f_k(x)$.

- $\hat{k}(x)$: the estimated classification of a given example. It is noted that $\hat{k}(x) = argmax_k( f_k(x) )$.

- $\hat{r}(x)$: the minimal perturbation for which $\hat{k}(x) \ne \hat{k}(x+\hat{r}(x))$.

The DeepFool algorithm only outputs the value of the minimal perturbation $\hat{r}(x)$ of a given example $x$.

Finally, the proposed formal definition for the robustness to adversarial examples of a given classifier is the expected value over the domain of examples for the norm of the minimal perturbation for an example divided by the norm of that same example. For practical purposes, the aforementioned expected value is approximated to the mean value for all examples in the available test domain of the classifier:

$\rho_{adv}(f) = 𝔼_X \frac{||\hat{r}(x)||_2}{||x||_2} ≈ \frac{1}{|T|} ∑_{x\in T} \frac{||\hat{r}(x)||_2}{||x||_2}$

Function to calculate the gradients of all the classes at the same time (with respect to the input)

In [3]:
import pickle
import pandas as pd
import numpy as np
import os
import librosa
import tensorflow as tf
import numpy as np
from copy import deepcopy
import keras
from tensorflow.keras import models, layers, regularizers, optimizers
from tensorflow.python.client import device_lib


In [4]:
def save_pkl(data, path):
    try:
        with open(path, "wb") as saved_data:
            pickle.dump(data, saved_data)
        saved_data.close()
    except:
        print('Fail to save data')

def load_pkl(path):
    try:
        with open(path, "rb") as loaded_data:
            to_return = pickle.load(loaded_data)
        loaded_data.close()
        return to_return
    except:
        print('Fail to load data')
        return None

In [55]:
def get_gradient(model, x, k):
    # Ensure the input is a tensor
    x = tf.convert_to_tensor(x, dtype=tf.float32)

    # Watch the input tensor for gradient calculation
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x)  # Watch the input tensor

        # Get the model's output
        results = model(x)

        # Ensure the output is the correct shape (batch_size, num_classes)
        results_k = results[0, k]  # Get the k-th class output for the first input in the batch

    # Compute the gradient of the k-th output with respect to the input
    gradients = tape.gradient(results_k, x)  # Gradient of k-th output wrt the input tensor

    del tape  # Free up the resources

    # Return gradients as numpy arrays
    return gradients.numpy(), results



Implementation of the deepfool algorithm

In [7]:
def example_robustness(x, r):
    """
    Calculate robustness measure for an individual example.
    """
    r_norm = np.sqrt(np.sum([np.linalg.norm(r_input)**2 for r_input in r]))
    x_norm = np.sqrt(np.sum([np.linalg.norm(x_input)**2 for x_input in x]))
    return r_norm / x_norm


def model_robustness(example_robustness_list):
    """
    Calculate mean and standard deviation of robustness for the model.
    """
    mean = np.mean(example_robustness_list)
    std = np.std(example_robustness_list)
    return mean, std

Running DeepFool for the RNN-LSTM

In [8]:
df = pd.read_csv("../UrbanSound8K/metadata/UrbanSound8K.csv")

In [21]:
def padding(path, duration = 4, sr = 44100):
    files = librosa.util.find_files(path)
    data = []

    for index, file_path in enumerate(files):
        try:
            audio, sr = librosa.load(file_path, sr=sr, mono=True)

            if len(audio) < duration*sr: # quando a duracao do audio for inferior aos 4 seg vamos adicionar padding
                audio = np.concatenate([audio,np.zeros(shape = (duration*sr - len(audio), ))])

            elif len(audio) > duration*sr: # quando a duracao do audio for superior aos 4 segs vamos reduzir a duracao
                audio = audio[:duration*sr]
        
            file_name = os.path.basename(file_path)
            data.append([file_name, audio])
        
        except Exception:
            print(f"Error in processig file {file_path}: {Exception}")

    return data

In [32]:
def feature_extraction(dataframe, audios):
    mfcc_coeficients = []
    labels = []

    for index in range(len(audios)):
        try:
            file_name =audios[index][0]
            if file_name:
                row = dataframe.loc[dataframe["slice_file_name"] == file_name]
                if not row.empty:
                    label = row.iloc[0,6] #get the corresponding label
                    # mfcc extraction
                    mfcc = librosa.feature.mfcc(
                        y = audios[index][1],
                        n_mfcc = 13, 
                        hop_length = 1024  
                    )
                    # mfcc normalization
                    mfcc = (mfcc - np.min(mfcc)) /( np.max(mfcc)- np.min(mfcc))
                    mfcc_coeficients.append(mfcc)
                    labels.append(label)
        except Exception:
            print(f"Error in processig file {audios[index][0]}: {Exception}")

    mfcc_coeficients = np.array(mfcc_coeficients)
    mfcc_coeficients = mfcc_coeficients.transpose(0,2,1) # transpose so it matches the input_shape of the LSTM
    labels = np.array(labels)
    return mfcc_coeficients, labels

In [23]:
fold_paths = ["../UrbanSound8K/audio/fold1",
              "../UrbanSound8K/audio/fold2",
              "../UrbanSound8K/audio/fold3",
              "../UrbanSound8K/audio/fold4",
              "../UrbanSound8K/audio/fold5",
              "../UrbanSound8K/audio/fold6",
              "../UrbanSound8K/audio/fold7",
              "../UrbanSound8K/audio/fold8",
              "../UrbanSound8K/audio/fold9",
              "../UrbanSound8K/audio/fold10"]

In [33]:
import gc
features = np.empty((10,), dtype=object)
label = np.empty((10,), dtype=object)

for i, fold in enumerate(fold_paths):
    print(f"Processing Fold Number {i+1}")
    audio_data = padding(fold)
    log_spectograms, labels = feature_extraction(df, audio_data)

    # normalizar os dados para estarem entre valores [0,1]
    log_spectograms_normalized = (log_spectograms - np.min(log_spectograms)) / (np.max(log_spectograms) - np.min(log_spectograms))

    # one-hot encoding dos labels
    encoded_labels = np.zeros((len(labels), 10))
    encoded_labels[np.arange(len(labels)), labels] = 1
    
    features[i] = log_spectograms_normalized
    label[i] = encoded_labels
    print("Features Shape: ",features[i].shape)
    print("Labels Shape: ",label[i].shape,"\n")
    
    del log_spectograms
    del log_spectograms_normalized
    del labels
    del encoded_labels
    gc.collect() # libertar memoria

save_pkl(features,"data/features_rnn.pkl")
save_pkl(label,"data/labels_rnn.pkl")

Processing Fold Number 1
Features Shape:  (873, 173, 13)
Labels Shape:  (873, 10) 

Processing Fold Number 2
Features Shape:  (888, 173, 13)
Labels Shape:  (888, 10) 

Processing Fold Number 3
Features Shape:  (925, 173, 13)
Labels Shape:  (925, 10) 

Processing Fold Number 4
Features Shape:  (990, 173, 13)
Labels Shape:  (990, 10) 

Processing Fold Number 5
Features Shape:  (936, 173, 13)
Labels Shape:  (936, 10) 

Processing Fold Number 6
Features Shape:  (823, 173, 13)
Labels Shape:  (823, 10) 

Processing Fold Number 7
Features Shape:  (838, 173, 13)
Labels Shape:  (838, 10) 

Processing Fold Number 8
Features Shape:  (806, 173, 13)
Labels Shape:  (806, 10) 

Processing Fold Number 9
Features Shape:  (816, 173, 13)
Labels Shape:  (816, 10) 

Processing Fold Number 10
Features Shape:  (837, 173, 13)
Labels Shape:  (837, 10) 



In [34]:
df_data = load_pkl("data/features_rnn.pkl")

In [35]:
isinstance(df_data[0][0], np.ndarray)

True

In [64]:
# function that implements the DeepFool algorithm
# model: the model to be used in the algorithm
# x0: the initial input without any perturbation
# eta: an overshoot value to be multiplied with each iteration's perturbation
# max_iter: maximum nummber of iterations  the algorithm is allowed to execute
def deepfool(model, x0, eta=0.01, max_iter=20):

    # obtain the initial estimated label
    f_x0 = model.predict(x0).flatten()
    print(f"f_x0 = {f_x0}")
    label_x0 = f_x0.argsort()[::-1][0]

    loop_i = 0
    xi = deepcopy(x0)
    label_xi = label_x0
    r = []

    # main loop
    while label_xi == label_x0 and loop_i < max_iter:
        w_l = [np.zeros(x_input.shape) for x_input in x0]
        f_l = 0
        fk_wk_min = np.inf
        grad_f_label_x0_on_xi, f_xi = get_gradient(model, xi, label_x0)

        for k in range(10): # k = 0, ..., 9 (possible classes in the problem considered for this project)
            if (k == label_x0):
                continue
            grad_f_k_on_xi, f_xi = get_gradient(model, xi, k)
            w_k = [g_f_k - g_f_label for g_f_k, g_f_label in zip(grad_f_k_on_xi, grad_f_label_x0_on_xi)]
            w_k_norm = np.sqrt(np.sum(np.fromiter([np.linalg.norm(w_k_input)**2 for w_k_input in w_k], dtype=np.float32)))
            f_k = f_xi[0,k] - f_xi[0,label_x0]
            fk_wk = np.linalg.norm(f_k) / (w_k_norm + 1e-3)
            if fk_wk < fk_wk_min:
                w_l, f_l = w_k, f_k
        
        w_l_squared_norm = np.sum(np.fromiter([np.linalg.norm(w_l_input)**2 for w_l_input in w_l], dtype=np.float32))
        f_l_norm = np.linalg.norm(f_l)
        ri_const = f_l_norm / (w_l_squared_norm + 1e-3)
        ri = [ri_const * w_l_input for w_l_input in w_l]
        r.append(ri)
        xi_new = [xi_item + (1+eta)*ri_item for xi_item, ri_item in zip(xi, ri)]
        xi = xi_new
        xi = np.array(xi)
        print(f"xi shape: {xi.shape}")
        label_xi = model.predict(xi).flatten()
        label_xi = label_xi.argsort()[::-1][0]
        loop_i += 1

    # main loop finished
    r_sum = [np.zeros(x_input.shape) for x_input in x0]
    for i in range(len(x0)):
        for r_i in r:
            r_sum[i] += r_i[i][0]

    # return the value of r(x), number of iterations performed, and the new label obtained by adding the perturbation to the input
    return r_sum, loop_i, label_xi


In [65]:
for f in range(1, 10+1):

    # this list will hold the values ||r(x)|| / ||x|| initially mentioned on the notebook 
    robustness_values_rnn_fold = []

    # utilize only the data of the f-th fold
    X_fold = df_data[f]

    
    

    # load the model from the performance evaluation cross validation run that used the f-th fold as the test fold
    # the compile keyword argument was set to false such that the model can be explicitly re-compiled using the compile settings defined on LSTM.ipynb
    fold_model_rnn = keras.models.load_model(f"kfold_metrics_LSTM/model_fold{f}.keras", compile=False)
    fold_model_rnn.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    # begin the run of each example in the f-th fold and the corresponding model on the DeepFool algorithm
    for i in range(len(X_fold)):
        print(f"Shape of X_fold[{i}]: {X_fold[i].shape}")  # Debugging
    
        example_input = np.expand_dims(X_fold[i], axis = 0)
        print(f"Shape of example_input: {example_input.shape}")

        # prepare the input for the implemented DeepFool function
        

        # run DeepFool
        # eta keyword argument was set to 10^6 due to the extremely small order of magnitude of each iteration's perturbation (between 10^-8 and 10^-5)
        perturbation, iters, fool_label = deepfool(fold_model_rnn, example_input, eta=1e6)

        # save the ||r(x)|| / ||x|| value into the list defined in the beginning of the "fold" for loop
        robustness_values_rnn_fold.append(example_robustness(example_input, perturbation))

    del X_fold
    gc.collect()

    # to save the results of each fold
    save_pkl(robustness_values_rnn_fold, f"robustness/robustness_values_rnn_fold{f}.pkl")
# check each folds' models results
for f in range(1, 10+1):
    robustness_values_rnn_fold = load_pkl(f"robustness/robustness_values_rnn_fold{f}.pkl")
    mean_robustness_rnn, std_robustness_rnn = model_robustness(robustness_values_rnn_fold)
    print(f"Fold {f} - The RNN model has a robustness of {mean_robustness_rnn: .7f} +/- {std_robustness_rnn: .7f}.")

Shape of X_fold[0]: (173, 13)
Shape of example_input: (1, 173, 13)
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
f_x0 = [0.05804431 0.00902075 0.07135397 0.5868831  0.01118041 0.06975934
 0.02193286 0.00964274 0.0830009  0.07918163]
xi shape: (1, 173, 13)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Shape of X_fold[1]: (173, 13)
Shape of example_input: (1, 173, 13)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
f_x0 = [0.07885264 0.01386668 0.12996542 0.43735862 0.0047444  0.07607406
 0.00988847 0.00529737 0.13397333 0.10997912]
xi shape: (1, 173, 13)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Shape of X_fold[2]: (173, 13)
Shape of example_input: (1, 173, 13)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
f_x0 = [0.04452556 0.0087773  0.06562144 0.66652155 0.00433099 0.05155517
 0.01938735 0.00304618 0.07900991 0.05722459]
xi shape: (1, 173, 13)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Shape of X_fold[3]: (173, 13)
Shape of example_input: (1, 173, 13)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
f_x0 = [0.05484143 0.01107342 0.08068

KeyboardInterrupt: 

In [52]:
for i in range(1,3):
    robustness_values_rnn_fold = load_pkl(f"robustness/robustness_values_rnn_fold{i}.pkl")
    mean_robustness_rnn, std_robustness_rnn = model_robustness(robustness_values_rnn_fold)
    print(f"Fold {i} - The RNN model has a robustness of {mean_robustness_rnn: .7f} +/- {std_robustness_rnn: .7f}.")

Fold 1 - The RNN model has a robustness of  0.0038434 +/-  0.0162903.
Fold 2 - The RNN model has a robustness of  0.0056982 +/-  0.0103444.


## References

- Moosavi-Dezfooli, S.-M., Fawzi, A., Frossard, P., Polytechnique, E. and De Lausanne, F. (2016). DeepFool: a simple and accurate method to fool deep neural networks: https://openaccess.thecvf.com/content_cvpr_2016/papers/Moosavi-Dezfooli_DeepFool_A_Simple_CVPR_2016_paper.pdf.

- TensorFlow. (2023). Introduction to gradients and automatic differentiation | TensorFlow Core. [online] Available at: https://www.tensorflow.org/guide/autodiff.